In [105]:
# !pip install google-search-results
import pandas as pd
api_key = "e4fb6e74a565645df616278e366b117d5d63cc04d23182ce7fca8c25ec2fda1b"
api = 'apify_api_BqCUn7b2esAHLm3kQPVRagWEykcmMT3u8Gv7'

In [106]:
from serpapi import GoogleSearch
import json

In [123]:
search_term = "Frontend developer job in India"
search_location = "India"
params = {
    "api_key" : api_key,
    "device" : "desktop",
    "engine" : "google_jobs",
    "q" : search_term,
    "hl": 'en',
    'gl': 'in',
    "start" : 0 #pagination
}

search = GoogleSearch(params)

In [124]:
data = search.get_dict()

https://serpapi.com/search


In [125]:
data.keys()

dict_keys(['search_metadata', 'search_parameters', 'jobs_results', 'chips'])

In [120]:
# with open(r"C:\Users\Jadha\PycharmProjects\python-stuff\sample.json", "w") as outfile:
#     json.dump(data['jobs_results'], outfile)

KeyError: 'jobs_results'

In [127]:
len(data['jobs_results'])

10

In [128]:
data['jobs_results']

[{'title': 'Frontend and Backend Developer',
  'company_name': 'thebckspace',
  'location': 'Mumbai, Maharashtra',
  'via': 'via LinkedIn',
  'description': 'We have an existing and running website. Language used is python and the backend is odoo.\n\nHere is the link to the website :\nhttps://www.thebckspace.com/\n\nWe are looking for someone to help us maintain the website, like help updating a few things and resolving bugs.',
  'job_highlights': [{'items': ['We have an existing and running website. Language used is python and the backend is odoo.\n\nHere is the link to the website :\nhttps://www.thebckspace.com/\n\nWe are looking for someone to help us maintain the website, like help updating a few things and resolving bugs.']}],
  'related_links': [{'link': 'https://www.google.com/search?hl=en&gl=in&q=thebckspace&sa=X&ved=0ahUKEwjv6v_h36D9AhWBjYkEHYIuDhwQmJACCJII',
    'text': 'See web results for thebckspace'}],
  'extensions': ['19 hours ago'],
  'detected_extensions': {'posted_at

In [129]:
job = data['jobs_results'][0]

In [130]:
print(job.keys())

dict_keys(['title', 'company_name', 'location', 'via', 'description', 'job_highlights', 'related_links', 'extensions', 'detected_extensions', 'job_id'])


In [131]:
print(job['title'])
print(job['company_name'])
print(job['location'])
print(job['via'])
# print(job['description']) # contails all the textual stuff
print([highlight['title'] for highlight in job['job_highlights']]) # Short of description
print(job['related_links'])
print(job['extensions'])
print(job['detected_extensions'])
# job['job_id']

Frontend and Backend Developer
thebckspace
Mumbai, Maharashtra
via LinkedIn


KeyError: 'title'

In [ ]:
job = data['jobs_results'][4]

In [ ]:
job

In [132]:
print(job['title'])
print(job['company_name'])
print(job['location'])
print(job['via'])
# print(job['description']) # contails all the textual stuff
print([highlight['title'] for highlight in job['job_highlights']]) # Short of description
print(job['related_links'])
print(job['extensions'])
print(job['detected_extensions'])

Frontend and Backend Developer
thebckspace
Mumbai, Maharashtra
via LinkedIn


KeyError: 'title'

## Get Combine Json data for 5 pages for dataScience

In [133]:
metadata = []
results = []

for i in range(5):
    search = GoogleSearch({**params, 'start' : i*10})
    page_data = search.get_dict()
    metadata.append(page_data['search_metadata'])
    results += page_data['jobs_results']

https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search
https://serpapi.com/search


## Create CSV file
extra features can be added like `employee website`

`Key Points` - Some day ago | fulltime or PartTime | worktype

In [134]:
data = {
    'title': [],
    'company_name': [],
    'location': [],
    'from': [],
    'Benefits': [],
    'Qualifications': [],
    'Responsibilities': [],
}

highlight_keys = ['Benefits', 'Qualifications', 'Responsibilities']

for job in results:
    data['title'].append(job['title'])
    data['company_name'].append(job['company_name'])
    data['location'].append(job['location'])
    data['from'].append(job['via'])
    
    for k in highlight_keys:
        found = False
        for h in job['job_highlights']:
            if 'title' in h.keys() and k == h['title']:
                data[k].append(". ".join(h['items']))
                found = True
        if not found:
            data[k].append("")


In [135]:
df = pd.DataFrame.from_dict(data)
df.to_csv('surpapi_mrng_data.csv', index=False)

In [136]:
df

,title,company_name,location,from,Benefits,Qualifications,Responsibilities
0,Frontend and Backend Developer,thebckspace,"Mumbai, Maharashtra",via LinkedIn,,,
1,Frontend Developer- ReactJS -4y,Sure4job,"Mumbai, Maharashtra",via LinkedIn,,,
2,Senior Frontend Developer,LIQVD ASIA,"Mumbai, Maharashtra",via Foundit.in,,,
3,Sr. Frontend Developer : ReactJS,BillDesk,"Mumbai, Maharashtra",via LinkedIn,,,
4,Senior Frontend Developer,SoftPixel Solutions Pvt. Ltd.,"Mumbai, Maharashtra",via LinkedIn,,,
5,Java Frontend Developer,Genesis HR Services,"Mumbai, Maharashtra",via LinkedIn,,,
6,urgent opening for Frontend Developer,T & M Services Consulting Private Limited,"Mumbai, Maharashtra",via Foundit.in,,,
7,Front End Developer,Channel Marketing,"Mumbai, Maharashtra",via LinkedIn,,,
8,Java Frontend Developer,TIAA GBS India,"Mumbai, Maharashtra",via TIAA Careers,,,
9,Senior Frontend Developer,Carufus Technology Pvt. Ltd.,"Mumbai, Maharashtra",via LinkedIn,,,


# Apify API

In [5]:
from apify_client import ApifyClient
# Initialize the ApifyClient with your API token
client = ApifyClient(token=api)

In [145]:
# Prepare the actor input
run_input = {
    "queries": "Frontend Developer job in India",
    "maxPagesPerQuery": 100,
    "countryCode": "in",
    "languageCode" : "en",
    "locationUule" : r"w+CAIQICIFU3VyYXQ=",
    "radius" : 500,
}

# Run the actor and wait for it to finish
run = client.actor("dan.scraper/google-jobs-scraper").call(run_input=run_input)

# Fetch and print actor results from the run's dataset (if there are any)
for item in client.dataset(run["defaultDatasetId"]).iterate_items():
    print(item)


{'searchQuery': {'term': 'Frontend Developer job in India', 'page': 1, 'type': 'SEARCH', 'domain': 'google.co.in', 'countryCode': 'IN', 'languageCode': 'en', 'locationUule': 'w CAIQICIFU3VyYXQ=', 'resultsPerPage': 10}, 'url': 'https://www.google.co.in/search?hl=en&ibp=htl%3Bjobs&lrad=500&q=Frontend%20Developer%20job%20in%20India&uule=w+CAIQICIFU3VyYXQ=', 'hasNextPage': True, 'googleJobs': [{'title': 'Frontend Developer', 'companyName': 'Mobile Programming LLC', 'location': 'Bengaluru, Karnataka, India', 'via': 'via LinkedIn', 'description': 'Frontend Developer\n\nExperience:- 5+ years\n\nLocation:- Bengaluru\n\nRequired Skill Set:- HTML5, CSS, JavaScript, TypeScript, React, Angular, Vue, Bootstrap, Node.js, UI/UX, SQL.\n\nJob description:-\n\nCompetency:-\n• 5+ years of software engineering experience focused on web development.\n• Hands on experience in architecting large scale frontend applications.\n• Have detailed hands-on experience of cutting edge web technologies (HTML5, CSS, Ja

In [147]:
# Fetch and print actor results from the run's dataset (if there are any)
for item in client.dataset(run["defaultDatasetId"]).iterate_items():
    print(item.keys())
    print(len(item['googleJobs']))

dict_keys(['searchQuery', 'url', 'hasNextPage', 'googleJobs', 'categories'])
10
dict_keys(['searchQuery', 'url', 'hasNextPage', 'googleJobs', 'categories'])
10
dict_keys(['searchQuery', 'url', 'hasNextPage', 'googleJobs', 'categories'])
10
dict_keys(['searchQuery', 'url', 'hasNextPage', 'googleJobs', 'categories'])
10
dict_keys(['searchQuery', 'url', 'hasNextPage', 'googleJobs', 'categories'])
10
dict_keys(['searchQuery', 'url', 'hasNextPage', 'googleJobs'])
0


In [148]:
data = {
    'title': [],
    'company_name': [],
    'location': [],
    'from': [],
    'Benefits': [],
    'Qualifications': [],
    'Responsibilities': [],
    'thumbnail': [],
    'applyLink': [],
    'description': [],
}

highlight_keys = ['Benefits', 'Qualifications', 'Responsibilities']

for page in client.dataset(run["defaultDatasetId"]).iterate_items():
    page_data = page['googleJobs']
    for job in page_data:
#         if 'jobHighlights' not in job.keys():
#             continue
        data['title'].append(job['title'])
        data['company_name'].append(job['companyName'])
        data['location'].append(job['location'] if 'location' in job.keys() else "") 
        data['from'].append(job['via'])
#         print(job['applyLink']['link'])
        data['applyLink'].append(job['applyLink']['link'])
        data['thumbnail'].append(job['thumbnail'] if 'thumbnail' in job.keys() else "")
        data['description'].append(job['description'])
        for k in highlight_keys:
            if 'jobHighlights' in job.keys():
                found = False
                for h in job['jobHighlights']:
                    if 'title' in h.keys() and k == h['title']:
                        data[k].append(". ".join(h['items']))
                        found = True
                if not found:
                    data[k].append("")
            else:
                data[k].append("")


In [149]:
df = pd.DataFrame.from_dict(data)
print(len(df))
display(df.head())

50


,title,company_name,location,from,Benefits,Qualifications,Responsibilities,thumbnail,applyLink,description
0,Frontend Developer,Mobile Programming LLC,"Bengaluru, Karnataka, India",via LinkedIn,,,,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://in.linkedin.com/jobs/view/frontend-dev...,Frontend Developer\n\nExperience:- 5+ years\n\...
1,Lead Frontend Developer,Infra.Market,"Bengaluru, Karnataka, India",via Bigshyft.com,,,,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://www.bigshyft.com/lead-frontend-develop...,What makes you a great fit:\n• Good problem so...
2,Frontend Developer,Assiduus Global,"Bengaluru, Karnataka, India",via Instahyre,,,,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://www.instahyre.com/job-233814-frontend-...,Assiduus Global is looking for ambitious and e...
3,Frontend Developer,QueueBuster POS,"Noida, Uttar Pradesh, India",via Bigshyft.com,,,,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://www.bigshyft.com/frontend-developer-jo...,What you’ll do:\n• Responsible for profiling a...
4,Sr Frontend Developer,Rock Interview,"Bengaluru, Karnataka, India",via LinkedIn,,,,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://in.linkedin.com/jobs/view/sr-frontend-...,Job Description:\n\nWe are looking for a Front...


In [150]:
df['company_name'].nunique()

45

In [151]:
df.to_csv("data_science_job_data_with_applylink_frontend.csv", index=False)